<a href="https://colab.research.google.com/github/xanasa14/MLImplementations/blob/master/NaiveBayesFromOfflineDatasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:


import numpy as np, pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer 

from sklearn.feature_extraction.text import CountVectorizer 

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')


w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [18]:
#importing our cancer dataset
#dataset = pd.read_csv('/content/Errors.csv')
#/content/fruitsBi.csv

dataset = pd.read_csv('/content/fruitsBi.csv')
#X = dataset['radius mean', 'texture mean', 'perimeter mean', 'area_mean','smoothness_mean','compactness mean','concavity mean','concave points mean','symmetry mean','fractal dimension mean','radius se' ]
col_list = ['id', 'label', 'Description']
#df = pd.read_csv("/content/Errors.csv", usecols=col_list)#dataset['diagnosis']
df = pd.read_csv("/content/fruitsBi.csv", usecols=col_list)#dataset['diagnosis']
#Setting our X and Y for trainig and testing 

#Dat  = df['Description']
#Y = dataset['label']
#print(type(Dat))
#print(Dat[0])



In [19]:
import spacy

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nlp = spacy.load('en', disable=['parser', 'ner'])

stop_words = stopwords.words('english')
#print(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer 


nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner']) 
import string  

#Removing punctuationes and those characters
def remove_punctuations(text):
  for punctuation in string.punctuation:
      text = text.replace(punctuation, '')
  return text
# Lowering Text from DataFrame
def loweringText(text):
    text = text.lower()
    return text
def convert(lst): 
    return ([i for item in lst for i in item.split()]) 
def remove_StopWords(text):
  line = text.split()
  text = ""
  for word in line:
    if(word not in stop_words):
      text += word
      text += " "
  return text

def lemmatize(text):
  line = text.split()
  txt = ""
  for word in line:
    doc = nlp(word)
    for token in doc:
      txt += token.lemma_
      txt += " "
  return txt

  # Extract the lemma for each token and join
  #print(line)
  #> 'the strip bat be hang on -PRON- foot for good'
  return line 
df["Desc"] = df['Description'].apply(remove_punctuations).apply(loweringText).apply(remove_StopWords).apply(lemmatize)
#df ["D"] = df["Desc"].apply(lemmatize)
#df.authors.apply(eval).apply(len).sum()

Dat  = df['Desc']
Y = dataset['label']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
cv=CountVectorizer() 
 
# this steps generates word counts for the words in your docs 
word_count_vector=cv.fit_transform(Dat)
word_count_vector.shape

(31, 3)

In [27]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(word_count_vector)
# print idf values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"]) 
 
# sort ascending 
df_idf.sort_values(by=['idf_weights'])

#Compute the TFIDF score for your documents

# count matrix 
count_vector=cv.transform(Dat) 
 
# tf-idf scores 
tf_idf_vector=tfidf_transformer.transform(count_vector)


In [28]:
feature_names = cv.get_feature_names() 
 
#get tfidf vector for first document 
first_document_vector=tf_idf_vector[0] 
 
#print the scores 
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
red,1.0
color,0.0
yellow,0.0


In [7]:
X_train, X_test, y_train, y_test = train_test_split(Dat, Y,test_size=0.5, random_state =0  )
#_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.2, random_state =0  )

print("\nX_train:\n")
print(X_train.head())
print(X_train.shape)

print("\nX_test:\n")
print(X_test.head())
print(X_test.shape)

print("\nX_train:\n")
print(y_train.head())
print(y_train.shape)

print("\nX_test:\n")
print(y_test.head())
print(y_test.shape)


X_train:

1              red 
30             red 
6     yellow color 
4              red 
18    yellow color 
Name: Desc, dtype: object
(15,)

X_test:

2              red 
29             red 
13             red 
10    yellow color 
27             red 
Name: Desc, dtype: object
(16,)

X_train:

1      apple
30     apple
6     banana
4      apple
18    banana
Name: label, dtype: object
(15,)

X_test:

2      apple
29     apple
13     apple
10    banana
27     apple
Name: label, dtype: object
(16,)


In [8]:
# Build the model
model = make_pipeline(TfidfVectorizer(), MultinomialNB())
# Train the model using the training data
model.fit(X_train, y_train)
# Predict the categories of the test data
predicted_categories = model.predict(Y)
print(predicted_categories)

['apple' 'apple' 'apple' 'apple' 'apple' 'apple' 'apple' 'apple' 'apple'
 'apple' 'apple' 'apple' 'apple' 'apple' 'apple' 'apple' 'apple' 'apple'
 'apple' 'apple' 'apple' 'apple' 'apple' 'apple' 'apple' 'apple' 'apple'
 'apple' 'apple' 'apple' 'apple']


In [9]:
print(predicted_categories)
print(Y)

['apple' 'apple' 'apple' 'apple' 'apple' 'apple' 'apple' 'apple' 'apple'
 'apple' 'apple' 'apple' 'apple' 'apple' 'apple' 'apple' 'apple' 'apple'
 'apple' 'apple' 'apple' 'apple' 'apple' 'apple' 'apple' 'apple' 'apple'
 'apple' 'apple' 'apple' 'apple']
0      apple
1      apple
2      apple
3      apple
4      apple
5     banana
6     banana
7     banana
8     banana
9     banana
10    banana
11    banana
12    banana
13     apple
14     apple
15     apple
16     apple
17     apple
18    banana
19    banana
20    banana
21    banana
22    banana
23    banana
24    banana
25    banana
26     apple
27     apple
28     apple
29     apple
30     apple
Name: label, dtype: object


In [10]:

print("The accuracy is {}".format(accuracy_score(predicted_categories, Y)))
#he accuracy is 0.7738980350504514
#The accuracy is 0.47058823529411764


The accuracy is 0.4838709677419355


In [11]:
confusion_matrix(Y, predicted_categories)


array([[15,  0],
       [16,  0]])

In [ ]:
# Starting with the CountVectorizer/TfidfTransformer approach...
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

cvec = CountVectorizer(stop_words='english', min_df=1, max_df=0.6, ngram_range=(1,2,))
cvec

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=0.6, max_features=None, min_df=1,
                ngram_range=(1, 2), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [ ]:
# Calculate all the n-grams found in all documents
from itertools import islice
cvec.fit(df.Desc)
list(islice(cvec.vocabulary_.items(), 20))

[('red', 1), ('yellow', 2), ('color', 0), ('yellow color', 3)]

In [ ]:
cvec_counts = cvec.transform(df.Desc)
print ('sparse matrix shape:', cvec_counts.shape)
print ('nonzero count:', cvec_counts.nnz)
print ('sparsity: %.2f%%' % (100.0 * cvec_counts.nnz / (cvec_counts.shape[0] * cvec_counts.shape[1])))

sparse matrix shape: (31, 4)
nonzero count: 63
sparsity: 50.81%


In [ ]:
occ = np.asarray(cvec_counts.sum(axis=0)).ravel().tolist()
counts_df = pd.DataFrame({'term': cvec.get_feature_names(), 'occurrences': occ})
counts_df.sort_values(by='occurrences', ascending=False).head(20)

,term,occurrences
0,color,16
2,yellow,16
3,yellow color,16
1,red,15


In [ ]:
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(cvec_counts)
transformed_weights

<31x4 sparse matrix of type '<class 'numpy.float64'>'
	with 63 stored elements in Compressed Sparse Row format>

In [ ]:
weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': cvec.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(20)

,term,weight
1,red,0.483871
0,color,0.297987
2,yellow,0.297987
3,yellow color,0.297987


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer(min_df=.1, max_df=0.6, stop_words='english', ngram_range=(1,2,))
tvec_weights = tvec.fit_transform(df.Desc.dropna())
weights = np.asarray(tvec_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': tvec.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(20)

,term,weight
1,red,0.483871
0,color,0.297987
2,yellow,0.297987
3,yellow color,0.297987


In [ ]:
print(df.Desc)

0              red 
1              red 
2              red 
3              red 
4              red 
5     yellow color 
6     yellow color 
7     yellow color 
8     yellow color 
9     yellow color 
10    yellow color 
11    yellow color 
12    yellow color 
13             red 
14             red 
15             red 
16             red 
17             red 
18    yellow color 
19    yellow color 
20    yellow color 
21    yellow color 
22    yellow color 
23    yellow color 
24    yellow color 
25    yellow color 
26             red 
27             red 
28             red 
29             red 
30             red 
Name: Desc, dtype: object


In [ ]:
weights_df

,term,weight
0,color,0.297987
1,red,0.483871
2,yellow,0.297987
3,yellow color,0.297987
